# Air Force Cadet Career Problem Walkthrough

This jupyter notebook demonstrates the implementation of the Air Force Cadet Career Problem (afccp) python module developed and maintained by Griffen Laird for his master's thesis. This notebook showcases the "CadetCareerProblem" class, which is a class object of these specific problem instances. This class pertains to specific instances of the AFSC matching problem (AFCCP) and there are numerous class methods pertaining to importing, exporting, solving, etc. Let's first discuss some preliminary material for this module.  

## Preliminaries

This module's main purpose is to automate as much of the matching process as possible. If we can focus on the general application of solving these problems, there is no reason that AFPC needs to spend so much time each year cleaning data and figuring out how to assign cadets to their AFSCs! That's where afccp comes in.

### globals.py

This script figures out the user's directory information and determines which python packages the user has access to. The majority of the packages I use are common python packages (pandas, numpy, matplotlib, etc.) but there are a few exceptions. The libraries pyomo (optimization package for python), sdv (synthetic data vault useful for generating realistic data --CTGAN is found here!), and sklearn (Sci-kit learn but it's version 0.0 since for whatever reason this is the one I found that performs MDS for the solution similarity plots) are some of the less common ones. This script checks these packages directly and prints out whether we have them or not.

In [1]:
from afccp.core.globals import *
print('current directory path:', dir_path)  # Directory path

ModuleNotFoundError: No module named 'afccp'

In [7]:
import os
dir_path = os.getcwd() + '/'
print(dir_path)

C:\Users\Griffen Laird\Desktop\Python Projects\afccp\afccp\executables\examples/


In [4]:
import sys
sys.path.append('../')